# Modelling a sapphire single-crystal filter with NCrystal

A sapphire single-crystal filter can be easily modelled in NCrystal, with a quick-and-dirty (and inexpensive) soluton being a cfg-string like `"stdlib::Al2O3_sg167_Corundum.ncmat;bragg=0;temp=200K"`. However, it might be interesting to actually model the Bragg diffraction with a single-crystal model. This notebook provides an interactive widget in which one can play with various single-crystal parameters and see the resulting transmission probabilities through the corresponding filter, and hopefully get a better idea about the correct settings needed for a particular application. For simplicity and speed the "crude energy smearing" used to smooth out the sharp bragg peaks in the plots, is simply a rolling average and not something more meaningful like a gaussian $\Delta\lambda/\lambda$ smearing.

Note that for reasons of efficiency, the plots in this notebook does not include the effects of multiple-scattering. In particular, the zig-zag effects which can to some degree enhance the transmission probability in single crystals is neglected. A more complete study would include such effects, and consequently increase the computational resources required - which is beyond the scope of this little notebook.

Refer to https://github.com/mctools/ncrystal/wiki/CfgRefDoc to learn more about specific cfg-string variables, like `bragg=0`, `sccutoff`, `dir1`, `dir2`, `dirtol`, and `mos`.

**WARNING: This notebook provides an interactive widget, which is not displayed when browsing the notebook on GitHub. You must download and open the notebook**

## Preamble
Install dependencies and prepare plots. Feel free to edit as you wish:

In [1]:
#By default we only do pip installs on Google Colab, but you
#can set the variable in the next line to True if you need it:
always_do_pip_installs = False
try:
  import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    from importlib.util import find_spec as _fs
    if not _fs('NCrystal'):
        %pip -q install ncrystal ipympl numpy matplotlib
#enable inline and interactive matplotlib plots:
if google_colab:
  google_colab.output.enable_custom_widget_manager()
%matplotlib ipympl
import matplotlib
matplotlib.rcParams.update({"figure.autolayout": True})
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  3009004, "too old NCrystal found"
NC.test() #< quick unit test that installation works

Tests completed succesfully


## Definitions
First a few definitions, affecting which options will be available in the widget further down.

In [2]:
material_file='stdlib::Al2O3_sg167_Corundum.ncmat'
#If not set, the sccutoff parameter defaults to 0.4 which might be a bit too crude. A value of 0.2 is most likely good enough.
sccutoffs = [0.2,0.0,0.4]
temperatures=[100,300]#Kelvin
bragg_modes = [ None, '100', '001' ]#which hkl plane to make parallel to the neutron (None means no Bragg diffraction at all)
mosaicities = ['1arcmin','5arcsec']
filter_thickness_cm = 7.5

## Interactive results
Just execute the following cell, then scroll down and play with the resulting widget below:

In [3]:
#Lots of code - just collapse this cell and play with the interactive widget it creates

import numpy as np
import matplotlib.pyplot as plt
NC.enableFactoryThreads()
neutron_energies = np.geomspace(1e-5,10.0,20000)#unit is eV, neutron energies used in plots

def tot_macro_xs( material, energies, direction=None ):
    from NCrystal.misc import MaterialSource
    mat = MaterialSource(material).load()
    xs = mat.scatter.xsect(ekin=energies,direction=direction)
    xa = mat.absorption.xsect(ekin=energies,direction=direction)
    return ( xs + xa )*mat.info.numberdensity

bmlist=[]
for temp in temperatures:
    for bm in bragg_modes:
        if bm is None:
            bmlist.append((f'bragg=0;temp={temp}',f'No Bragg diffraction, T={temp}K'))
        else:
            hkl=list(int(e) for e in bm)
            _  = 'dir1=@crys_hkl:%i,%i,%i@lab:0,0,1'%tuple(hkl)
            _ += ';dir2=@crys_hkl:%i,%i,%i@lab:1,1,0'%(hkl[1],hkl[2],hkl[0])#anything not parallel to hkl
            _ += ';dirtol=180deg'#needed since dir2 was not carefully calculated to be consistent
            _ += f';temp={temp}'
            for sccutoff in sccutoffs:
                for m in mosaicities:
                    bmlist.append((_+f';sccutoff={sccutoff};mos={m}',bm+f' (T={temp}K, mos={m}, sccutoff={sccutoff}Å)'))

_plot_cache={}
def _smear(c):
    #Extremely crude "running mean" smearing!!!!!
    nsmear=len(neutron_energies)//100
    return np.convolve(c.copy(),  np.ones(nsmear)/nsmear, mode='same')

def _get_xs( cfgstr, do_smear = False, do_ptransm=False ):
    key=(cfgstr,do_smear,do_ptransm)
    xs = _plot_cache.get(key)
    if xs is not None:
        return xs
    if do_ptransm:
        if do_smear:
            xs = _smear(_get_xs(cfgstr,False,True))
        else:
            xs = _get_xs(cfgstr,False,False)
            xs = 100.0*np.exp( -xs*filter_thickness_cm)        
    else:
        if do_smear:
            xs = _smear(_get_xs( cfgstr, False ))
        else:
            xs = tot_macro_xs(cfgstr,neutron_energies,direction=(0,0,1))
    #print("ADDING TO CACHE:",key)
    _plot_cache[key]=xs
    return xs

def do_plot( selected_lbls, do_smear, do_ptransm ):
    plt.clf()
    plt.semilogx()
    plt.xlabel('Neutron energy (eV)')
    for extracfg, lbl in bmlist:
        if not selected_lbls or lbl in selected_lbls:
            plt.plot( neutron_energies,
                      _get_xs(f'{material_file};{extracfg}',do_smear=do_smear,do_ptransm=do_ptransm),
                      label=lbl,
                      lw = 1 if not do_smear else None )
    if do_ptransm:
        plt.ylim(0,100)
        plt.ylabel(f'Transmission probability after {filter_thickness_cm}cm (%)')
    else:
        plt.semilogy()
        plt.ylabel('Macroscopic cross sections (1/cm)')
    plt.legend(fontsize='small')
    plt.grid()
    plt.show()

from ipywidgets import interact
lbls = [lbl for _,lbl in bmlist]
@interact( show_cross_sections = False,
           curve1=lbls,
           curve2=['none']+lbls,
           curve3=['none']+lbls,
           curve4=['none']+lbls,
           crude_energy_smear = True )
def interactive_plot( curve1=lbls[0],curve2=lbls[1],curve3='none',curve4='none',
                      crude_energy_smear=True, show_cross_sections = False ):
    lbls = [curve1,curve2,curve3,curve4]
    lbls = [e for e in lbls if e!='none']
    do_plot( lbls,
             do_smear = crude_energy_smear,
             do_ptransm = not show_cross_sections
           )
    def printcfgstr(curvename,lbl):
        if lbl=='none':
            return
        cp=[c for c,l in bmlist if l==lbl][0]
        print(f'\n{curvename} defined by cfgstr:\n\n   "{material_file};{cp}"')
    printcfgstr('curve1',curve1)
    printcfgstr('curve2',curve2)
    printcfgstr('curve3',curve3)
    printcfgstr('curve4',curve4)

    

interactive(children=(Dropdown(description='curve1', options=('No Bragg diffraction, T=100K', '100 (T=100K, mo…